# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [30]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Read the CSV and Perform Basic Data Cleaning

In [32]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [33]:
df=pd.get_dummies(df, columns=['home_ownership','verification_status','next_pymnt_d'])
label_encoder = LabelEncoder()
label_encoder.fit(df["pymnt_plan"])
df["pymnt_plan"]=label_encoder.transform(df["pymnt_plan"])

label_encoder.fit(df["initial_list_status"])
df["initial_list_status"]=label_encoder.transform(df["initial_list_status"])

label_encoder.fit(df["application_type"])
df["application_type"]=label_encoder.transform(df["application_type"])

label_encoder.fit(df["application_type"])
df["application_type"]=label_encoder.transform(df["application_type"])

label_encoder.fit(df["hardship_flag"])
df["hardship_flag"]=label_encoder.transform(df["hardship_flag"])

label_encoder.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"]=label_encoder.transform(df["debt_settlement_flag"])

df.drop(labels=['issue_d'],axis=1,inplace=True)
# Create our features
X = df.copy()
X.drop("loan_status",axis=1,inplace=True)

# Create our target
y = df["loan_status"]

In [34]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.0,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.383161,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.486161,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,4.000000,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [36]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=78)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [39]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [40]:
# Fit the Standard Scaler with the training data
X_scaler=scaler.fit(X_train)

In [41]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [68]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model = brf_model.fit(X_train_scaled, y_train)

In [69]:
# Calculated the balanced accuracy score
predictions = brf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.699279596746943

In [70]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,50,42
Actual 1,2480,14633


In [71]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.54      0.86      0.04      0.68      0.45        92
   low_risk       1.00      0.86      0.54      0.92      0.68      0.48     17113

avg / total       0.99      0.85      0.55      0.92      0.68      0.48     17205



In [72]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.07072338254204659, 'last_pymnt_amnt'),
 (0.06965655884581433, 'total_rec_prncp'),
 (0.05773421268467116, 'total_pymnt_inv'),
 (0.056513488460141655, 'total_pymnt'),
 (0.05037414064917492, 'total_rec_int'),
 (0.027593087461120925, 'int_rate'),
 (0.02068910344622258, 'dti'),
 (0.018660030235146344, 'mo_sin_old_rev_tl_op'),
 (0.017911624083260094, 'max_bal_bc'),
 (0.01760733704412738, 'annual_inc'),
 (0.01675095517941912, 'bc_util'),
 (0.016134262473500145, 'mo_sin_old_il_acct'),
 (0.015895703132011732, 'total_rev_hi_lim'),
 (0.01581900822462686, 'out_prncp_inv'),
 (0.015740906301961528, 'mths_since_recent_inq'),
 (0.01560067975073092, 'total_bc_limit'),
 (0.015534345506065576, 'revol_bal'),
 (0.01531162938828252, 'avg_cur_bal'),
 (0.014943917607213072, 'tot_hi_cred_lim'),
 (0.014936810113464358, 'bc_open_to_buy'),
 (0.014675780295679732, 'out_prncp'),
 (0.014264173693367205, 'installment'),
 (0.013613107063699642, 'mths_since_rcnt_il'),
 (0.013492713083808417, 'tot_cur_bal'),
 (0.013

### Easy Ensemble Classifier

In [73]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec=EasyEnsembleClassifier(n_estimators=100,random_state=1)
eec.fit(X_train,y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [74]:
# Calculated the balanced accuracy score
predictions = brf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.699279596746943

In [75]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,50,42
Actual 1,2480,14633


In [76]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.54      0.86      0.04      0.68      0.45        92
   low_risk       1.00      0.86      0.54      0.92      0.68      0.48     17113

avg / total       0.99      0.85      0.55      0.92      0.68      0.48     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    EasyEnsemble classifier

2. Which model had the best recall score?

    both are the same

3. Which model had the best geometric mean score?

    both are the same.

4. What are the top three features?

     'last_pymnt_amnt','total_rec_prncp', 'total_pymnt_inv'